In [1]:
import anndata
import numpy as np
import scanpy as sc
sc.settings.verbosity = 2
import openTSNE
from readcount_tools import readcount_pipeline

In [2]:
%%time
#30sec
adata_qumi_census_readcounts = anndata.read_h5ad('data/tasic/adata_qumi_census.h5ad')
adata_qumi_census_readcounts.var_names_make_unique()

tcmalloc: large alloc 1545371648 bytes == 0x63ed6000 @ 
tcmalloc: large alloc 1545371648 bytes == 0xee9f8000 @ 
tcmalloc: large alloc 1545371648 bytes == 0x17937a000 @ 
tcmalloc: large alloc 5305827328 bytes == 0x20400a000 @ 


CPU times: user 1.73 s, sys: 7.2 s, total: 8.93 s
Wall time: 19.9 s


/usr/local/lib/python3.8/dist-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/usr/local/lib/python3.8/dist-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
adata_qumis = adata_qumi_census_readcounts.copy()
adata_qumis.X = adata_qumi_census_readcounts.layers['qumis'].copy()
del adata_qumis.layers

adata_census = adata_qumi_census_readcounts.copy()
adata_census.X = adata_qumi_census_readcounts.layers['census'].copy()
del adata_census.layers

adata_readcounts = adata_qumi_census_readcounts.copy()
adata_readcounts.X = adata_qumi_census_readcounts.layers['readcounts'].copy()
del adata_readcounts.layers
adata_readcount_umi_processing = adata_readcounts.copy()

tcmalloc: large alloc 5305827328 bytes == 0x4e223c000 @ 
tcmalloc: large alloc 5305827328 bytes == 0x4b4158000 @ 
tcmalloc: large alloc 5305827328 bytes == 0x4b4158000 @ 


In [4]:
n_hvgs=3000
n_comps=1000

In [5]:
def umi_pipeline(adata,n_comps,n_hvgs):
        sc.pp.filter_genes(adata,min_cells=5)        
        sc.experimental.pp.recipe_pearson_residuals(adata,n_comps=n_comps,n_top_genes=n_hvgs)
        tsne = openTSNE.TSNE(random_state=42,verbose=True,n_jobs=38)
        adata.obsm['tsne'] = np.array(tsne.fit(X=adata.obsm['X_pca']))

In [6]:
%%time
for adata in [adata_census,adata_qumis]:
    umi_pipeline(adata,n_comps=n_comps,n_hvgs=n_hvgs)

filtered out 2593 genes that are detected in less than 5 cells
extracting highly variable genes


/gpfs01/berens/user/jlause/.local/lib/python3.8/site-packages/scanpy/experimental/pp/_highly_variable_genes.py:73: UserWarning: `flavor='pearson_residuals'` expects raw count data, but non-integers were found.
  warnings.warn(


computing analytic Pearson residuals on adata.X


/gpfs01/berens/user/jlause/.local/lib/python3.8/site-packages/scanpy/experimental/pp/_normalization.py:43: UserWarning: `normalize_pearson_residuals()` expects raw count data, but non-integers were found.
  warn(


    finished (0:00:02)
computing PCA
    on highly variable genes
    with n_comps=1000
    finished (0:01:56)
--------------------------------------------------------------------------------
TSNE(n_jobs=38, random_state=42, verbose=True)
--------------------------------------------------------------------------------
===> Finding 90 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 19.63 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.37 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 0.68 seconds
===> Running optimization with exaggeration=12.00, lr=1985.17 for 250 iterations...
Iteration   50, KL divergence 5.4157, 50 iterations in 3.3952 sec
Iteration  100, KL divergence 4.4559, 50 iterations in 3.3226 sec
Iteration  150, KL divergence 4.2254, 50 iterations in 3.1868 sec
Iteration  200, KL divergence 4.1156, 50 iterations in 1.2663 sec
Iteration  250, KL divergence 4.0524, 50 iterations in 

In [7]:
%%time
readcount_pipeline(adata_readcounts,alpha=50,theta=100, n_hvgs=n_hvgs,n_comps=n_comps)

filtered out 2593 genes that are detected in less than 5 cells
pr_theta100_alpha50.0


tcmalloc: large alloc 4811669504 bytes == 0x7ebf28000 @ 
tcmalloc: large alloc 4811669504 bytes == 0x99569c000 @ 
tcmalloc: large alloc 4811669504 bytes == 0xab4360000 @ 
tcmalloc: large alloc 4811669504 bytes == 0x99569c000 @ 
tcmalloc: large alloc 4811669504 bytes == 0xbd3824000 @ 
tcmalloc: large alloc 4811669504 bytes == 0xcf24e8000 @ 


pr_theta100_alpha50.0_afterHVG
--------------------------------------------------------------------------------
TSNE(n_jobs=38, random_state=42, verbose=True)
--------------------------------------------------------------------------------
===> Finding 90 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 6.65 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.22 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 0.93 seconds
===> Running optimization with exaggeration=12.00, lr=1985.17 for 250 iterations...
Iteration   50, KL divergence 5.8027, 50 iterations in 1.7785 sec
Iteration  100, KL divergence 4.5160, 50 iterations in 1.6048 sec
Iteration  150, KL divergence 4.2299, 50 iterations in 1.4758 sec
Iteration  200, KL divergence 4.0997, 50 iterations in 1.4825 sec
Iteration  250, KL divergence 4.0235, 50 iterations in 1.5101 sec
   --> Time elapsed: 7.85 seconds
===> Running optimization with exagg

AnnData object with n_obs × n_vars = 23822 × 3000
    obs: 'clusters', 'class', 'non_neuronal_idx', 'n_genes_by_counts_allGenes', 'total_counts_allGenes', 'clustercolor', 'sample_id', 'subclass', 'clustername'
    var: 'genes', 'gene_id', 'chromosome', 'gene_name', 'n_cells_by_counts_allCells', 'mean_counts_allCells', 'pct_dropout_by_counts_allCells', 'total_counts_allCells', 'n_cells_allCells', 'marker_idx', 'name', 'symbol', 'type_of_gene', 'ensembl.type_of_gene', 'coarse_types', 'gene_var_allCells', 'gene_mean_allCells', 'gene_FF_allCells', 'gene_fraction_zeros_allCells', 'transcript_len_max', 'n_transcripts', 'gene_entrez_id', 'ensembl_gene', 'n_cells', 'pr_theta100_alpha50.0_var', 'top3000_pr_theta100_alpha50.0', 'means', 'pr_theta100_alpha50.0_afterHVG_var'
    uns: 'clustercolors', 'clusternames', 'markernames', 'cpr_alpha', 'cpr_theta'
    obsm: 'pca1000', 'tsne'
    layers: 'pr_theta100_alpha50.0', 'pr_theta100_alpha50.0_afterHVG'

In [8]:
#save
adata_qumis.write_h5ad('data/tasic/qumi_census_comparion/adata_qumis_processed.h5ad')
adata_census.write_h5ad('data/tasic/qumi_census_comparion/adata_census_processed.h5ad')
adata_readcounts.write_h5ad('data/tasic/qumi_census_comparion/adata_readcounts_processed.h5ad')